Download one of the small web graph dataset from 
https://snap.stanford.edu/data/web-Stanford.html

We have discussed a basic pagerank algorithm in class based on eigenvector computation. Read Chapter 4,5 of the reference and perform page ranking by construction Google pagerank matrix for the downloaded data set.


In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
f = open('/content/drive/MyDrive/IR/web-Stanford.txt')
data=f.readlines()
# print(f.readlines())

# Page Rank Algorithm

In [24]:
# here we are storing the data inside a variable
ndata=data[4:len(data)]

In [26]:
# printing out the data below to show that it was stored in the variable
# ndata

In [5]:
# installing a package for visual representation of graphs
!pip3 install networkx

In [27]:
# import csv
# i=0
# with open('/content/drive/MyDrive/IR/web-Stanford.txt', 'r') as file:
#     reader = csv.reader(file,delimiter = '\t')
#     for row in reader:
#         print(row)
#         i+=1
#         if i>4:
#           break

# above code is just to view the file content and then remove the header content
# in the below code

['# Directed graph (each unordered pair of nodes is saved once): web-Stanford.txt ']
['# Stanford web graph from 2002']
['# Nodes: 281903 Edges: 2312497']
['# FromNodeId', 'ToNodeId']
['1', '6548']


In [29]:
import networkx as nx
import csv
i=0

pageG=nx.DiGraph()

with open('/content/drive/MyDrive/IR/web-Stanford.txt', 'r') as file:
    reader = csv.reader(file,delimiter = '\t')
    for row in reader:
      if i>4:
        pageG.add_edge(row[0],row[1])
      i+=1


In [30]:
pageG.is_directed() 
# verifying that the graph generated from the dataset is directed graph.

True

In [31]:
print("Nodes:",len(list(pageG.nodes)))
print("Edges:",len(list(pageG.edges)))
# viewing the nodes and edges in the generated graph.

Nodes: 281903
Edges: 2312496


In [32]:
def pagerank(G, alpha=0.85, personalization=None,
			max_iter=100, tol=1.0e-6, nstart=None, weight='weight',
			dangling=None):
	"""Return the PageRank of the nodes in the graph.

	PageRank computes a ranking of the nodes in the graph G based on
	the structure of the incoming links. It was originally designed as
	an algorithm to rank web pages.

	Parameters
	----------
	G : graph
	A NetworkX graph. Undirected graphs will be converted to a directed
	graph with two directed edges for each undirected edge.

	alpha : float, optional
	Damping parameter for PageRank, default=0.85.

	personalization: dict, optional
	The "personalization vector" consisting of a dictionary with a
	key for every graph node and nonzero personalization value for each node.
	By default, a uniform distribution is used.

	max_iter : integer, optional
	Maximum number of iterations in power method eigenvalue solver.

	tol : float, optional
	Error tolerance used to check convergence in power method solver.

	nstart : dictionary, optional
	Starting value of PageRank iteration for each node.

	weight : key, optional
	Edge data key to use as weight. If None weights are set to 1.

	dangling: dict, optional
	The outedges to be assigned to any "dangling" nodes, i.e., nodes without
	any outedges. The dict key is the node the outedge points to and the dict
	value is the weight of that outedge. By default, dangling nodes are given
	outedges according to the personalization vector (uniform if not
	specified). This must be selected to result in an irreducible transition
	matrix (see notes under google_matrix). It may be common to have the
	dangling dict to be the same as the personalization dict.

	Returns
	-------
	pagerank : dictionary
	Dictionary of nodes with PageRank as value

	Notes
	-----
	The eigenvector calculation is done by the power iteration method
	and has no guarantee of convergence. The iteration will stop
	after max_iter iterations or an error tolerance of
	number_of_nodes(G)*tol has been reached.

	The PageRank algorithm was designed for directed graphs but this
	algorithm does not check if the input graph is directed and will
	execute on undirected graphs by converting each edge in the
	directed graph to two edges.

	
	"""
	if len(G) == 0:
		return {}

	if not G.is_directed():
		D = G.to_directed()
	else:
		D = G

	# Create a copy in (right) stochastic form
	W = nx.stochastic_graph(D, weight=weight) #A right-stochastic graph is a weighted digraph in which for each node, the sum of the weights of all the out-edges of that node is 1
	N = W.number_of_nodes()

	# Choose fixed starting vector if not given
	if nstart is None:
		x = dict.fromkeys(W, 1.0 / N)
	else:
		# Normalized nstart vector
		s = float(sum(nstart.values()))
		x = dict((k, v / s) for k, v in nstart.items())

	if personalization is None:

		# Assign uniform personalization vector if not given
		p = dict.fromkeys(W, 1.0 / N)
	else:
		missing = set(G) - set(personalization)
		if missing:
			raise NetworkXError('Personalization dictionary '
								'must have a value for every node. '
								'Missing nodes %s' % missing)
		s = float(sum(personalization.values()))
		p = dict((k, v / s) for k, v in personalization.items())

	if dangling is None:

		# Use personalization vector if dangling vector not specified
		dangling_weights = p
	else:
		missing = set(G) - set(dangling)
		if missing:
			raise NetworkXError('Dangling node dictionary '
								'must have a value for every node. '
								'Missing nodes %s' % missing)
		s = float(sum(dangling.values()))
		dangling_weights = dict((k, v/s) for k, v in dangling.items())
	dangling_nodes = [n for n in W if W.out_degree(n, weight=weight) == 0.0]

	# power iteration: make up to max_iter iterations
	for _ in range(max_iter):
		xlast = x
		x = dict.fromkeys(xlast.keys(), 0)
		danglesum = alpha * sum(xlast[n] for n in dangling_nodes)
		for n in x:

			# this matrix multiply looks odd because it is
			# doing a left multiply x^T=xlast^T*W
			for nbr in W[n]:
				x[nbr] += alpha * xlast[n] * W[n][nbr][weight]
			
			x[n] += danglesum * dangling_weights[n] + (1.0 - alpha) * p[n]


		# check convergence, l1 norm
		err = sum([abs(x[n] - xlast[n]) for n in x])
		if err < N*tol:
			return x
	raise NetworkXError('pagerank: power iteration failed to converge '
						'in %d iterations.' % max_iter)


In [33]:
pr=nx.pagerank(pageG,0.5)

In [34]:
import operator
sorted_d = dict( sorted(pr.items(), key=operator.itemgetter(1),reverse=True))
sorted_d
# displaying the sorted dictionary (or map) of nodes and its corresponding probability (or page rank score)
# We can see the highest relevant node is 226411 which has the highest probability

{'226411': 0.00918893329549712,
 '241454': 0.00672366095565256,
 '89073': 0.0064866444127989475,
 '67756': 0.003650422078568506,
 '69358': 0.0033769724639487785,
 '186750': 0.002771901241071356,
 '225872': 0.0027693944545717224,
 '231363': 0.002714677953917289,
 '134832': 0.0025239606121161364,
 '234704': 0.0023929250180366483,
 '105607': 0.0020248873410027074,
 '38342': 0.0018845855293060042,
 '167295': 0.0018505692069031153,
 '198090': 0.0016838393413581584,
 '81435': 0.0016776832511666217,
 '214128': 0.0016774558783924842,
 '34573': 0.0016390493879623283,
 '93778': 0.0015859170115298633,
 '245659': 0.0015814310183366194,
 '136821': 0.0015534382307778253,
 '68889': 0.001543832416304476,
 '119479': 0.0015117085324740682,
 '95163': 0.0014065549381793686,
 '251796': 0.0014065549381793686,
 '272442': 0.0014065549381793686,
 '158568': 0.0011736670932553203,
 '101161': 0.0011649378652433457,
 '258348': 0.0009758417217183442,
 '60210': 0.0009689871013393683,
 '117152': 0.0008796454506242434

In [ ]:
# showing that the sum of the probabilities (page rank scores) of all nodes is equal to 1.
import numpy as np
np.sum(list(sorted_d.values()))

1.0000000000000029